In [23]:
%matplotlib inline

Plot OpenFOAM data {#openfoam_example}
==================


In [24]:
from __future__ import annotations

import pyvista
from pyvista import examples

This example uses data from a lid-driven cavity flow. It is recommended
to use `pyvista.POpenFOAMReader`{.interpreted-text role="class"} for
reading OpenFOAM files for more control over reading data.

This example will only run correctly in versions of vtk\>=9.1.0. The
names of the patch arrays and resulting keys in the read mesh will be
different in prior versions.


In [25]:
filename = examples.download_cavity(load=False)
reader = pyvista.POpenFOAMReader(filename)
print(filename)

/home/jdark/.cache/pyvista_3/OpenFOAM.zip.unzip/OpenFOAM/cavity/case.foam


OpenFOAM datasets include multiple sub-datasets including the internal
mesh and patches, typically boundaries. This can be inspected before
reading the data.


In [26]:
print(f"All patch names: {reader.patch_array_names}")
print(f"All patch status: {reader.all_patch_arrays_status}")

All patch names: ['internalMesh', 'patch/movingWall', 'patch/fixedWalls', 'patch/frontAndBack']
All patch status: {'internalMesh': True, 'patch/movingWall': True, 'patch/fixedWalls': True, 'patch/frontAndBack': True}


This data is represented as a `pyvista.MultiBlock`{.interpreted-text
role="class"} object. The internal mesh will be located in the top-level
MultiBlock mesh.


In [27]:
mesh = reader.read()
print(f"Mesh patches: {mesh.keys()}")
internal_mesh = mesh["internalMesh"]  # or internal_mesh = mesh[0]

Mesh patches: ['internalMesh', 'boundary']


In this case the internal mesh is a
`pyvista.UnstructuredGrid`{.interpreted-text role="class"}.


In [28]:
print(internal_mesh)

UnstructuredGrid (0x7f2e25e24520)
  N Cells:    400
  N Points:   882
  X Bounds:   0.000e+00, 1.000e-01
  Y Bounds:   0.000e+00, 1.000e-01
  Z Bounds:   0.000e+00, 1.000e-02
  N Arrays:   4


Additional Patch meshes are nested inside another MultiBlock mesh. The
name of the sub-level MultiBlock mesh depends on the vtk version.


In [29]:
boundaries = mesh["boundary"]
print(boundaries)
print(f"Boundaries patches: {boundaries.keys()}")
print(boundaries["movingWall"])

MultiBlock (0x7f2e25e24160)
  N Blocks    3
  X Bounds    0.000, 0.100
  Y Bounds    0.000, 0.100
  Z Bounds    0.000, 0.010
Boundaries patches: ['movingWall', 'fixedWalls', 'frontAndBack']
PolyData (0x7f2e25dfae00)
  N Cells:    20
  N Points:   42
  N Strips:   0
  X Bounds:   0.000e+00, 1.000e-01
  Y Bounds:   1.000e-01, 1.000e-01
  Z Bounds:   0.000e+00, 1.000e-02
  N Arrays:   4


The default in OpenFOAMReader is to translate the existing cell data to
point data. Therefore, the cell data arrays are duplicated in point
data.


In [30]:
print("Cell Data:")
print(internal_mesh.cell_data)
print("\nPoint Data:")
print(internal_mesh.point_data)

Cell Data:
pyvista DataSetAttributes
Association     : CELL
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (400, 3)             VECTORS
    p                       float32    (400,)               SCALARS

Point Data:
pyvista DataSetAttributes
Association     : POINT
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (882, 3)             VECTORS
    p                       float32    (882,)               SCALARS


This behavior can be turned off if only cell data is required.


In [31]:
reader.cell_to_point_creation = False
internal_mesh = reader.read()["internalMesh"]
print("Cell Data:")
print(internal_mesh.cell_data)
print("\nPoint Data:")
print(internal_mesh.point_data)

Cell Data:
pyvista DataSetAttributes
Association     : CELL
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (400, 3)             VECTORS
    p                       float32    (400,)               SCALARS

Point Data:
pyvista DataSetAttributes
Association     : POINT
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays : None


Now we will read in all the data at the last time point.


In [32]:
print(f"Available Time Values: {reader.time_values}")
reader.set_active_time_value(2.5)
reader.cell_to_point_creation = True  # Need point data for streamlines
mesh = reader.read()
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

Available Time Values: [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]


This OpenFOAM simulation is in 3D with only 1 cell in the z-direction.
First, the solution is sliced in the center of the z-direction.
`pyvista.DataSetFilters.streamlines_evenly_spaced_2D`{.interpreted-text
role="func"} requires the data to lie in the z=0 plane. So, after the
domain sliced, it is translated to `z=0`.


In [33]:
def slice_z_center(mesh):
    """Slice mesh through center in z normal direction, move to z=0."""
    slice_mesh = mesh.slice(normal='z')
    slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh


slice_internal_mesh = slice_z_center(internal_mesh)
slice_boundaries = pyvista.MultiBlock(
    {key: slice_z_center(boundaries[key]) for key in boundaries.keys()},
)

Streamlines are generated using the point data \"U\".


In [34]:
# streamlines = slice_internal_mesh.streamlines_evenly_spaced_2D(
#     vectors='U',
#     start_position=(0.05, 0.05, 0),
#     separating_distance=1,
#     separating_distance_ratio=0.1,
# )

Plot streamlines colored by velocity magnitude. Additionally, the moving
and fixed wall boundaries are plotted.


In [35]:
# plotter = pyvista.Plotter()
# plotter.add_mesh(slice_boundaries["movingWall"], color='red', line_width=3)
# plotter.add_mesh(slice_boundaries["fixedWalls"], color='black', line_width=3)
# plotter.add_mesh(streamlines.tube(radius=0.0005), scalars="U")
# plotter.view_xy()
# plotter.enable_parallel_projection()
# plotter.show()

In [36]:
print(internal_mesh.point_data.keys())  # Lists available point-based fields
print(internal_mesh.cell_data.keys())   # Lists available cell-based fields


['U', 'p']
['U', 'p']


In [37]:
cell_data = internal_mesh.cell_data["U"]  # Example for cell_data
cell_data

pyvista_ndarray([[ 2.53494e-04, -2.50378e-04,  0.00000e+00],
                 [ 1.41338e-04,  1.11475e-04,  0.00000e+00],
                 [-1.17699e-03,  5.64654e-04,  0.00000e+00],
                 ...,
                 [ 5.83952e-01, -7.26601e-02,  0.00000e+00],
                 [ 4.15827e-01, -1.27868e-01,  0.00000e+00],
                 [ 3.08819e-01, -1.49468e-01,  0.00000e+00]],
                shape=(400, 3), dtype=float32)

In [38]:
print(internal_mesh.cells_dict.keys())


dict_keys([np.uint8(12)])


In [39]:
from dolfinx import mesh as dmesh
import numpy as np
from mpi4py import MPI

# Extract points and connectivity
points = internal_mesh.points
cells = internal_mesh.cells_dict  # Dictionary mapping cell type to connectivity

# Assume tetrahedral mesh (modify based on actual mesh type)
hex_cells = cells.get(12)  # 12 corresponds to VTK_HEXAHEDRON
if hex_cells is None:
    raise ValueError("No hexahedron cells found in the mesh.")

## Connectivity of the mesh (topology) - The second dimension indicates the type of cell used
args_conn = np.argsort(hex_cells, axis=1)
rows = np.arange(hex_cells.shape[0])[:, None]
connectivity = hex_cells[rows, args_conn]

import ufl
# Define mesh element
shape = "hexahedron"
degree = 1
cell = ufl.Cell(shape)

import basix
v_cg = basix.ufl.element(
    "Lagrange", cell.cellname(), 1, shape=(3,)
)
mesh_ufl = ufl.Mesh(v_cg)


In [40]:
print(hex_cells.shape)

(400, 8)


In [41]:
# Create Dolfinx Mesh
domain = dmesh.create_mesh(MPI.COMM_WORLD, connectivity, points, mesh_ufl)
domain.topology.index_map(domain.topology.dim).size_global

400

In [ ]:
import dolfinx
v_dg = basix.ufl.element(
    "DG", cell.cellname(), 0, shape=(3,)
)
V = dolfinx.fem.functionspace(domain, v_dg)
u = dolfinx.fem.Function(V)

In [ ]:
u.x.array[:] = internal_mesh.cell_data["U"][domain.topology.original_cell_index].flatten()

print(domain.topology.original_cell_index)
print(domain.topology.original_cell_index.shape)

[  0   1  20   2  21  40   3  22  41  60   4  23  42  61  80   5  24  43
  62  81 100   6  25  44  63  82 101 120   7  26  45  64  83 102 121 140
   8  27  46  65  84 103 122 141 160   9  28  47  66  85 104 123 142 161
 180  10  29  48  67  86 105 124 143 162 181 200  11  30  49  68  87 106
 125 144 163 182 201 220  12  31  50  69  88 107 126 145 164 183 202 221
 240  13  32  51  70  89 108 127 146 165 184 203 222 241 260  14  33  52
  71  90 109 128 147 166 185 204 223 242 261 280  15  34  53  72  91 110
 129 148 167 186 205 224 243 262 281 300  16  35  54  73  92 111 130 149
 168 187 206 225 244 263 282 301 320  17  36  55  74  93 112 131 150 169
 188 207 226 245 264 283 302 321 340  18  37  56  75  94 113 132 151 170
 189 208 227 246 265 284 303 322 341 360  19  38  57  76  95 114 133 152
 171 190 209 228 247 266 285 304 323 342 361 380  39  58  77  96 115 134
 153 172 191 210 229 248 267 286 305 324 343 362 381  59  78  97 116 135
 154 173 192 211 230 249 268 287 306 325 344 363 38

In [52]:
internal_mesh.point_data["U"]

pyvista_ndarray([[0. , 0. , 0. ],
                 [0. , 0. , 0. ],
                 [0. , 0. , 0. ],
                 ...,
                 [1. , 0. , 0. ],
                 [1. , 0. , 0. ],
                 [0.5, 0. , 0. ]], shape=(882, 3), dtype=float32)

In [51]:
writer = dolfinx.io.VTXWriter(MPI.COMM_WORLD, "velocity.bp", u, engine="BP5")
writer.write(t=0)